In [1]:
# DB → cmds 파일 생성 (재사용 가능 버전)
import json
import pymysql

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "crawling"

# ==================== 설정 영역 (여기를 수정하세요) ====================
# 게시판 타입 (필터링용)
BOARD_TYPE = "docForm"  # ✨ 추가: hredu, cp, sales 등

# DB 테이블명 (이제 고정)
TABLE_NAME = "board_posts"

# 출력 파일명
OUTPUT_FILE = f"posts_{BOARD_TYPE}.cmds"  # ✨ 자동으로 타입 반영

# ======================================================================

def generate_cmds_file(table_name, board_type, output_file):
    """DB에서 특정 type의 데이터를 읽어 cmds 파일 생성"""
    
    print(f"📂 DB 연결 중...")
    print(f"📂 테이블: {table_name}")
    print(f"📂 게시판 타입: {board_type}")  # ✨ 추가
    print(f"📂 출력 파일: {output_file}\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 type의 데이터만 읽기
            query = f"SELECT * FROM {table_name} WHERE type = %s"  # ✨ WHERE 조건 추가
            cursor.execute(query, (board_type,))
            posts = cursor.fetchall()
            
            total_count = len(posts)
            print(f"✅ 총 {total_count}개의 posts 발견\n")
            
            # 2. cmds 파일 생성
            with open(output_file, 'w', encoding='utf-8') as f:
                for idx, post in enumerate(posts, 1):
                    try:
                        # attaches_json을 파싱하여 attaches 배열로 변환
                        if post.get('attaches_json'):
                            attaches = json.loads(post['attaches_json'])
                        else:
                            attaches = []
                        
                        # cmds 형식에 맞는 JSON 객체 생성
                        post_data = {
                            "sourceId": post.get('sourceId', ''),
                            "targetBoardId": post.get('targetBoardId', ''),
                            "createdAt": post.get('createdAt', ''),
                            "updatedAt": post.get('updatedAt', ''),
                            "inlineHandling": post.get('inlineHandling', ''),
                            "writer": {
                                "emailId": post.get('writer_emailId', '')
                            },
                            "title": post.get('title', ''),
                            "readCount": post.get('readCount', ''),
                            "content": post.get('content', ''),
                            "status": post.get('status', ''),
                            "attaches": attaches
                        }
                        
                        # addPost{JSON} 형식으로 작성
                        line = 'addPost' + json.dumps(post_data, ensure_ascii=False)
                        f.write(line + '\n')
                        
                        if idx % 10 == 0:
                            print(f"   처리 중... {idx}/{total_count}")
                        
                    except Exception as e:
                        print(f"❌ [Post {idx}] 처리 오류: {e}")
                        continue
            
            print(f"\n{'='*60}")
            print(f"✅ cmds 파일 생성 완료!")
            print(f"   - 게시판 타입: {board_type}")  # ✨ 추가
            print(f"   - 총 게시글: {total_count}개")
            print(f"   - 파일 위치: {output_file}")
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")

# 실행
if __name__ == "__main__":
    generate_cmds_file(TABLE_NAME, BOARD_TYPE, OUTPUT_FILE)  # ✨ board_type 인자 추가

📂 DB 연결 중...
📂 테이블: board_posts
📂 게시판 타입: docForm
📂 출력 파일: posts_docForm.cmds

✅ 총 78개의 posts 발견

   처리 중... 10/78
   처리 중... 20/78
   처리 중... 30/78
   처리 중... 40/78
   처리 중... 50/78
   처리 중... 60/78
   처리 중... 70/78

✅ cmds 파일 생성 완료!
   - 게시판 타입: docForm
   - 총 게시글: 78개
   - 파일 위치: posts_docForm.cmds

✅ DB 연결 종료
